<hr>

<h1>0. Import libraries </h1>

<p> 0. Import all the necessary libraries </p>

In [223]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import os
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import string
from nltk.stem.porter import *    
import re
from forex_python.converter import CurrencyRates
from decimal import Decimal
import numpy as np
import functools
import json
import math
import heapq
from heapq import heappush, heappop 
from keplergl import KeplerGl
import geopandas as gpd

from  geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="NPC12345")

c = CurrencyRates()

stemmer = PorterStemmer()

# nltk.download("stopwords") only the first time
# nltk.download("punkt")

url = 'https://www.findamasters.com/masters-degrees/msc-degrees/'


<hr>

<h1> 1. Data collection </h1>

For this homework, there is no provided dataset. Instead, you have to build your own. Your search engine will run on text documents. So, here
we detail the procedure to follow for the data collection. We strongly suggest you work on different modules when implementing the required functions. For example, you may have a ```crawler.py``` module, a ```parser.py``` module, and a ```engine.py``` module: this is a good practice that improves readability in reporting and efficiency in deploying the code. Be careful; you are likely dealing with exceptions and other possible issues! 


<h5 style="color:green"> 1.1 Get the list of master's degree courses </h5>

We start with the list of courses to include in your corpus of documents. In particular, we focus on web scrapping the [MSc Degrees](https://www.findamasters.com/masters-degrees/msc-degrees/). Next, we want you to **collect the URL** associated with each site in the list from the previously collected list.
The list is long and split into many pages. Therefore, we ask you to retrieve only the URLs of the places listed in **the first 400 pages** (each page has 15 courses, so you will end up with 6000 unique master's degree URLs).

The output of this step is a `.txt` file whose single line corresponds to the master's URL.

In [224]:
"""
Takes in input a page URL and a path
and saves the master's links of the URL in the file at path
"""
def getMasterLinks(siteUrl, filePath):
    result_url = requests.get(siteUrl)                                                      # fetch page
    result_soup = BeautifulSoup(result_url.text)                                            # parse page using a html interpreter
    result_links = result_soup.find_all('a', {'class': 'courseLink'})                       # get the master links from page
    lines = ["https://www.findamasters.com"+item['href']+"\n" for item in result_links]     # add the URL root header to each link
    fd = open(filePath, 'a')                                                                # open the links file in append mode
    fd.writelines(lines)                                                                    # append links to the file 
    fd.close()                                                                              # close the file 


"""
Given a range of pages the function parses all the master
links from each page (there are 15 master link per page).
and saves them all in a single file.
"""
def getMasterLinksInPages(startPage, endPage):
    for i in range(startPage, endPage+1):                                                   # iterate on pages indexes
        pageUrl = url+"?PG="+str(i)                                                         # create page url
        getMasterLinks(pageUrl, "data/links.txt")                                           # get the masters links from page 
        time.sleep(1)                                                                       # wait a second (to not overload server)

# getMasterLinksInPages(1, 400)



<h5 style="color:green"> 1.2 Crawl master's degree pages </h5>

Once you get all the URLs in the first 400 pages of the list, you:

1. Download the HTML corresponding to each of the collected URLs.
2. After you collect a single page, immediately save its `HTML` in a file. In this way, if your program stops for any reason, you will not lose the data collected up to the stopping point.
3. Organize the downloaded `HTML` pages into folders. Each folder will contain the `HTML` of the courses on page 1, page 2, ... of the list of master's programs.
   
__Tip__: Due to the large number of pages you should download, you can use some methods that can help you shorten the time. If you employed a particular process or approach, kindly describe it.

In [225]:

"""
Takes a list of links and a path to a folder
load the pages of the links 
and save them in the folder 
"""

def getHTMLs(links, htmlSaveFolder):
    os.mkdir(htmlSaveFolder)                                    # create the folder
    for i, link in enumerate(links):                            # iterate on links
        htmlDoc = requests.get(link).text                       # load the page (using the link)
        while(len(htmlDoc) < 7000):                             # while the page is a "Just a moment" page
            time.sleep(60)                                      # wait a minute (let the server rest)
            htmlDoc = requests.get(link).text                   # load the page again
        filename = str(i+1)+"_"+link.split("/")[5]+".html"      # create the path for the new file
        htmlFile = open(htmlSaveFolder + "/" + filename, "w")   # create the new file
        htmlFile.write(htmlDoc)                                 # save the contents
        htmlFile.close()                                        # close the file
        time.sleep(0.1)                                         # wait 0.1 sec (to not overwhelm the server)



"""
Load the masters from page [firstPage] to page
[lastPage]. The function iterates on grups of 15 links 
(assuming that a masters page contains exactly 15 masters)
loads the 15 masters pages and saves them in a directory.
"""

def loadMasters(firstPage, lastPage):
    fd = open("data/links.txt", "r")                                            # open the links file
    links = fd.read().splitlines()                                              # get the links list
    for i in range((firstPage-1)*15, (lastPage)*15, 15):                        # for each masters page
        getHTMLs(links[i:i+15], "data/htmls/Page_" + str((i//15) + 1))          # get the htmls of the 15 masters in the page
                                                                                # and save them in a directory Page_[pageNumber]

# loadMasters(1, 400)  # do not uncomment  it takes hours to load all the pages

<h5 style="color:green">1.3 Parse downloaded pages </h5>

At this point, you should have all the HTML documents about the master's degree of interest, and you can start to extract specific information. The list of the information we desire for each course and their format is as follows:

1. Course Name (to save as ```courseName```): string;
2. University (to save as ```universityName```): string;
3. Faculty (to save as ```facultyName```): string
4. Full or Part Time (to save as ```isItFullTime```): string;
5. Short Description (to save as ```description```): string;
6. Start Date (to save as ```startDate```): string;
7. Fees (to save as ```fees```): string;
8. Modality (to save as ```modality```):string;
9. Duration (to save as ```duration```):string;
10. City (to save as ```city```): string;
11. Country (to save as ```country```): string;
12. Presence or online modality (to save as ```administration```): string;
13. Link to the page (to save as ```url```): string.
    
For each master's degree, you create a `course_i.tsv` file of this structure:

```
courseName \t universityName \t  ... \t url
```

If an information is missing, you just leave it as an empty string.

In [226]:

"""
 The function takes in input a range of folders [startFolder, endFolder] inside [foldersLocation] 
each containing 15 master pages. For each folder in [foldersLocation] the function iterates on
masters docs extracting the following information:

courseName
universityName
facultyName
isItFullTime
description
startDate
fees
modality
duration
city
country
administration
ulr

It finds the url using the links inside [linksFile]
considering that the masters pages where loaded in the
same order as the links.

Finally it saves the extracted information as a tsv file
inside [fileName] 

"""
def makeTSV(startFolder, endFolder, foldersLocation, linksFile, fileName):

    with open(linksFile, "r") as links:             # get the links from [linksFile]
        links = linksFile.read().splitlines()        
    
    tsvFile = open(fileName, "a")                   # create a tsvFile and open in append mode
    headers = [                                     # define the headers of the tsv table
        "courseName",
        "universityName",
        "facultyName",
        "isItFullTime",
        "description",
        "startDate",
        "fees",
        "modality",
        "duration",
        "city",
        "country",
        "administration",
        "url"
    ]
    tsvFile.write("\t".join(headers)+"\n")          # save the headers in the file
    
    
    
    for folderIndex in range(startFolder, endFolder+1):                                     # Iterate on folders in [foldersLocation]
        path = foldersLocation+"/Page_"+str(folderIndex)                                    # get the current folder path 
        fileNames = sorted(os.listdir(path), key=lambda x: int(x.split("_")[0]))            # get the doc names inside the folder (sorted by index)
        for i, fileName in enumerate(fileNames):                                            # iterate on docs inside the folder
            filePath = path+"/"+fileName                                                    # get the current doc path
            fd = open(filePath, 'r')                                                        # open the current doc 
            page = BeautifulSoup(fd.read())                                                 # parse the current doc using a html interpreter
    
            courseName = page.find('h1', {'class': 'course-header__course-title'}).text     # get the courseName from doc
            universityName = page.find('a', {'class': 'course-header__institution'}).text   # get the universityName from doc
            facultyName = page.find('a', {'class': 'course-header__department'}).text       # get the facultyName from doc 
            isItFullTime = page.find('a', {'class': 'concealLink'}).text                    # get the isItFullTime from doc
            description =  " ".join(map(lambda x: x.text, page.find('div', {'id': 'Snippet'}).contents)) #get the description from doc
            startDate = page.find('span', {'class': 'key-info__start-date'}).text           # get the startDate from doc
    
            fees = page.find('div', {'class': 'course-sections__fees'})                     # get the the fees from doc
            fees = fees.find('p').text if fees else "NAN"                                   # if no such info then use "NAN"
    
            modality = page.find_all('a', {'class': 'concealLink'})[1].text                 # get the modality from doc
            duration = page.find('span', {'class': 'key-info__duration'}).text              # get duration from doc
            city = page.find('a', {'class': 'course-data__city'}).text                      # get the city form doc
            country = page.find('a', {'class': 'course-data__country'}).text                # get the country from doc
    
            administration = page.find('a', {'class': 'course-data__on-campus'})            # get the administration from doc
            administration = administration.text if administration else "NAN"               # if no such info then use "NAN"
    
            url = links[(folderIndex-1)*15 + i]                                             # get the link from the links list
                                                                                            # considering the current folder and doc
    
            record = "\t".join([                                                            # save the extracted information in a record of \t divided values
                courseName.replace('\t', ' ', -1), 
                universityName.replace('\t', ' ', -1), 
                facultyName.replace('\t', ' ', -1), 
                isItFullTime.replace('\t', ' ', -1), 
                description.replace('\t', ' ', -1), 
                startDate.replace('\t', ' ', -1), 
                fees.replace('\t', ' ', -1), 
                modality.replace('\t', ' ', -1), 
                duration.replace('\t', ' ', -1), 
                city.replace('\t', ' ', -1), 
                country.replace('\t', ' ', -1), 
                administration.replace('\t', ' ', -1), 
                url.replace('\t', ' ', -1)
            ]).replace("\n", ' ', -1)
    
            tsvFile.write(record+"\n")                                                      # append the record to the tsvFile
            fd.close()                                                                      # close the doc
        
    tsvFile.close()                                                                         # close the tsvFile (after parsing all docs)




# call the function and generate tsv
startFolder = 1 
endFolder = 400
foldersLocation = "data/htmls"
linksLocation = "data/links.txt"
tsvFile = "data/masters.tsv"
# makeTSV(startFolder, endFolder, foldersLocation, linksLocation, tsvFile)


In [227]:
df = pd.read_csv('data/masters.tsv', sep="\t", header=0)  # check if the tsv file is ok
df

,courseName,universityName,facultyName,isItFullTime,description,startDate,fees,modality,duration,city,country,administration,url
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,School of Engineering and Built Environment,Full time,3D visualisation and animation play a role i...,September,Please see the university website for further ...,MSc,1 year full-time,Glasgow,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
1,Accounting and Finance - MSc,University of Leeds,Leeds University Business School,Full time,Businesses and governments rely on sound fin...,September,"UK: £18,000 (Total) International: £34,750 (To...",MSc,1 year full time,Leeds,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
2,Accounting and Finance (MSc),University of Bath,School of Management,Full time,Develop in-depth knowledge of accounting and...,September,Please see the university website for further ...,MSc,1 year full-time,Bath,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
3,"Accounting, Accountability & Financial Manag...",King’s College London,King’s Business School,Full time,"Our Accounting, Accountability & Financial M...",September,Please see the university website for further ...,MSc,1 year FT,London,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
4,"Accounting, Financial Management and Digital...",University of Reading,Henley Business School,Full time,Embark on a professional accounting career w...,September,Please see the university website for further ...,MSc,1 year full time,Reading,United Kingdom,On Campus,https://www.findamasters.com/masters-degrees/c...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Masters Of Finance (International Finance),Zhejiang Gongshan University,International Programs,Full time,Master’s in Finance (International Finance) ...,September,US$3500 per year,MA,2.5 Years,Hangzhou,China,On Campus,https://www.findamasters.com/masters-degrees/c...
5996,Master's of Financial Technology (Fintech),Harbour.Space University,Masters Programmes,Full time,Harbour.Space's FinTech Master programme is ...,"September, January","€29,900/year",MSc,1 Year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/c...
5997,Master's of Front-end Development,Harbour.Space University,Masters Programmes,Full time,Front-end Development at Harbour.Space Unive...,"September, January","€29,900/year",MSc,1 year,Barcelona,Spain,On Campus,https://www.findamasters.com/masters-degrees/c...
5998,Masters of Science in Business,Oregon State University,School of Business,Full time,Our Master of Science in Business (MSB) will...,See Course,You can find more information here: Please see...,Part time,12 months,Corvallis,USA,NAN,https://www.findamasters.com/masters-degrees/c...


<hr>

<h1> 2. Search Engine </h1>
<p> Now, we want to create two different Search Engines that, given as input a query, return the courses that match the query. </p>

<h5 style="color:green"> 2.1 Preprocessing </h5>


<b> 2.1.1 Preprocessing the text </b>
<p>First, you must pre-process all the information collected for each MSc by:

1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed
   
For this purpose, you can use the [`nltk library](https://www.nltk.org/).
</p>


In [228]:

"""
Tokenize the description 
select only the words that are not in stopWords and contains only letters
set words to lower case and stem them
"""
def preprocess(text:str, stopWords:set):
    return [stemmer.stem(word.lower()) for word in word_tokenize(text) if (word.lower() not in stopWords) and word.isalpha()]


"""
    1.Create a new dataframe where the preprocessed text will be
stored as [tokens] 

    2.Preprocess the description for each master and 
store them in the new dataframe 

    3.Set an incremental id for the new dataframe
it wil be usefull in the next step to build the reverse index
"""
pdf = df.loc[:, ['courseName', 'universityName', 'description', 'city', 'country', 'fees']]
wordsToRemove = set(stopwords.words('english')).union(string.punctuation)
pdf.loc[:, 'tokens'] = df.description.map(lambda x: preprocess(x, wordsToRemove))
pdf.loc[:, 'id'] = pd.Series(range(0, pdf.size))
pdf

,courseName,universityName,description,city,country,fees,tokens,id
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,3D visualisation and animation play a role i...,Glasgow,United Kingdom,Please see the university website for further ...,"[visualis, anim, play, role, mani, area, popul...",0
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fin...,Leeds,United Kingdom,"UK: £18,000 (Total) International: £34,750 (To...","[busi, govern, reli, sound, financi, knowledg,...",1
2,Accounting and Finance (MSc),University of Bath,Develop in-depth knowledge of accounting and...,Bath,United Kingdom,Please see the university website for further ...,"[develop, knowledg, account, financ, theori, l...",2
3,"Accounting, Accountability & Financial Manag...",King’s College London,"Our Accounting, Accountability & Financial M...",London,United Kingdom,Please see the university website for further ...,"[account, account, financi, manag, msc, cours,...",3
4,"Accounting, Financial Management and Digital...",University of Reading,Embark on a professional accounting career w...,Reading,United Kingdom,Please see the university website for further ...,"[embark, profession, account, career, academ, ...",4
...,...,...,...,...,...,...,...,...
5995,Masters Of Finance (International Finance),Zhejiang Gongshan University,Master’s in Finance (International Finance) ...,Hangzhou,China,US$3500 per year,"[master, financ, intern, financ, zhejiang, gon...",5995
5996,Master's of Financial Technology (Fintech),Harbour.Space University,Harbour.Space's FinTech Master programme is ...,Barcelona,Spain,"€29,900/year","[fintech, master, programm, design, prepar, gr...",5996
5997,Master's of Front-end Development,Harbour.Space University,Front-end Development at Harbour.Space Unive...,Barcelona,Spain,"€29,900/year","[develop, univers, provid, uniqu, environ, stu...",5997
5998,Masters of Science in Business,Oregon State University,Our Master of Science in Business (MSB) will...,Corvallis,USA,You can find more information here: Please see...,"[master, scienc, busi, msb, give, focus, explo...",5998


<b> 2.1.2 Preprocessing the fees column </b>

Moreover, we want the field ```fees``` to collect numeric information. As you will see, you scraped textual information for this attribute in the dataset: sketch whatever method you need (using regex, for example, to find currency symbol) to collect information and, in case of multiple information, retrieve only the highest fees. Finally, once you have collected numerical information, you likely will have different currencies: this can be chaotic, so let chatGPT guide you in the choice and deployment of an API to convert this column to a common currency of your choice (it can be USD, EUR or whatever you want). Ultimately, you will have a ```float``` column renamed ```fees (CHOSEN COMMON CURRENCY)```.

In [229]:

"""
Load a currency dataset. It's used to get currency code given a currency symbol.
Source: https://gist.github.com/ksafranski/2973986
"""
currencies = pd.read_json('data/Common-Currency.json', convert_axes=3)
currencies

,symbol,name,symbol_native,decimal_digits,rounding,code,name_plural
0,$,US Dollar,$,2,0.0,USD,US dollars
1,CA$,Canadian Dollar,$,2,0.0,CAD,Canadian dollars
2,€,Euro,€,2,0.0,EUR,euros
3,AED,United Arab Emirates Dirham,د.إ.‏,2,0.0,AED,UAE dirhams
4,Af,Afghan Afghani,؋,0,0.0,AFN,Afghan Afghanis
...,...,...,...,...,...,...,...
114,CFA,CFA Franc BCEAO,CFA,0,0.0,XOF,CFA francs BCEAO
115,YR,Yemeni Rial,ر.ي.‏,0,0.0,YER,Yemeni rials
116,R,South African Rand,R,2,0.0,ZAR,South African rand
117,ZK,Zambian Kwacha,ZK,0,0.0,ZMK,Zambian kwachas


In [230]:
"""
Load a countries dataset. It's used to get a currency code given a country name.
Source: https://gist.github.com/tiagodealmeida/0b97ccf117252d742dddf098bc6cc58a
"""
countries = pd.read_json('data/countries.json')
countries

,countryCode,countryName,currencyCode,population,capital,continentName
0,AD,Andorra,EUR,84000,Andorra la Vella,Europe
1,AE,United Arab Emirates,AED,4975593,Abu Dhabi,Asia
2,AF,Afghanistan,AFN,29121286,Kabul,Asia
3,AG,Antigua and Barbuda,XCD,86754,St. John's,North America
4,AI,Anguilla,XCD,13254,The Valley,North America
...,...,...,...,...,...,...
245,YE,Yemen,YER,23495361,Sanaa,Asia
246,YT,Mayotte,EUR,159042,Mamoudzou,Africa
247,ZA,South Africa,ZAR,49000000,Pretoria,Africa
248,ZM,Zambia,ZMW,13460305,Lusaka,Africa


In [255]:

"""
    Preprocess the fees

    There are different notations for the fees

    Case1) Most fees have currency symbol in front of them
        $ 123.31
        € 34,000

    Case2) Some fees have the currency code in front of the ammount
        ISK 1234
        EUR 20,000 

    Case3) Some fees have the currency code after the ammount 
        75 EUR
        120.00 USD

    We must avoid ambiguity with currency. Let's say we have the following
    scenario:

    fees: $20,000    country: Canada    ($ = CA$)
    fees: $35,000    country: USA       ($ = US$)

    We can fix this by selecting all the masters that
    contain the $ symbol in description. Replace $ with the currency
    code instead (we must consider the country field to get the right currency code)

    After this we can math the patterns using regex expressions
    we can use a different regex for each pattern and then merge
    the results together.

    So we need a function that takes in input a text like
     "xxxxxxxx  $10,000  xxxxxx 23 000 EUR  xxxxxx  ISK 45,000"

     and returns only the ammount with the currency codes
    ("USD", 10000)
    ("EUR", 23000)
    ("ISK", 45000)

    After this we need a function that converts different currency ammounts to USD 
    so we can get the following result
    ("USD", 10000)
    ("USD", 24456.12)
    ("USD", 21023.22)

    Then we can easily pick the largest ammount and save it
    in a [fees_USD] field
    
"""

dollarCountriesCodes = currencies.loc[currencies.symbol_native == "$", 'code']                        # Get the dollar country codes
dollarCountriesDf = countries.loc[countries.currencyCode.isin(dollarCountriesCodes.values)]           # Get the dollar countries rows from countries
dollarCountries = set(dollarCountriesDf.countryName.to_list())                                        # Get the dollar countries names

usdRates:dict = c.get_rates('USD')                                                                    # Get the exchange rates for USD 
                                                                                                      # necessary to transform other currencies values in dollars
usdRates['USD'] = 1                                                                                   # set the exchange of USD/USD to 1 (not present by default)

currencySymbols = set(currencies.symbol.to_list())                                                    # get a set of currency symbols $,€,£ ...
currencyCodes = set(currencies.code.to_list())                                                        # get a set of currency codes  USD, EUR, GBL

symbolsRe = "|".join(map(lambda x: f"[{x}]" if len(x) == 1 else x ,currencySymbols))                  # regular expression pattern for matching currency symbols $|€|£ 
codesRe = "|".join(map(lambda x: f"[{x}]" if len(x) == 1 else x ,currencyCodes))                      # regular expression pattern for matching currency codes USD|EUR|GBL




"""
Function takes in input a string containing 0 or more
money ammounts and return a list of touples 
 (currencyCode,  ammountValue)
Example
 "xxxxxxxx  $10,000  xxxxxx 23 000 EUR  xxxxxx  ISK 45,000"
("USD", 10000)
("EUR", 23000)
("ISK", 45000)
"""
def parseFees(fees:str, country:str):

    if country in dollarCountries:                                                                   # If a country is a "dollarCountry"
        countryCurrencyCode = countries.loc[countries.countryName == country].currencyCode.values    # get the country currencyCode
        if len(countryCurrencyCode) > 0: fees = fees.replace("$", countryCurrencyCode[0], -1)        # replace $ with country currencyCode

    l = []                                                                                           # prepare an empty list for the results

                                                                                                     # Lets consider the 3 forms in which we can find an ammount

                                                                                                     # Case 1  -  [symbol value]   ex  $450,000
    regex1 = re.compile(f"({symbolsRe}) *(([0-9]{3}[, .])*[0-9]+(.[0-9]+)?)")                        # define regular expression to match fees in string
    matches1 = re.finditer(regex1, fees)                                                             # find all matches
    for mtch in matches1:                                                                            # iterate on matches in string
        symbol = mtch.group(1)                                                                       # get the symbol
        if symbol in currencySymbols:                                                                # if symbol is valid
            ammount = int(float(mtch.group(2).replace(',', '').replace(' ', '').replace('.', '')))   # get the ammount (lean the value of [,. ])
            code = currencies.loc[currencies.symbol == symbol].code.values[0]                        # use the symbol to get the currency code
            l.append((code, ammount))                                                                # append the tuple to the results
    
                                                                                                     # Case 2 - [currencyCode value] ex ISK 25,500
    regex2 = re.compile(f"({codesRe}) *(([0-9]{3}[, .])*[0-9]+(.[0-9]+)?)")                          # define regular expression to match fees in string
    matches2 = re.finditer(regex2, fees)                                                             # find all matches
    for mtch in matches2:                                                                            # iterate on matches in string
        ammount = int(float(mtch.group(2).replace(',', '').replace(' ', '').replace('.', '')))       # get the ammount 
        code = mtch.group(1)                                                                         # get the currency code 
        l.append((code, ammount))                                                                    # append the tuple to the results
                                                                                                      
                                                                                                     # Case 3 - [value currencyCode]  ex  75 EUR
    regex3 = re.compile(f"(([0-9]{3}[, .])*[0-9]+(.[0-9]+)?) +({codesRe})")                          # define regular expression to match fees in string
    matches3 = re.finditer(regex3, fees)                                                             # find all matches
    for mtch in matches3:                                                                            # iterate on matches in string
        ammount = int(float(mtch.group(1).replace(',', '').replace(' ', '').replace('.', '')))       # get the ammount 
        code = mtch.group(4)                                                                         # get the currency code 
        l.append((code, ammount))                                                                    # append the tuple to the results

    return l                                                                                         # return the results



"""
Return the max ammount in fees string 
converted to US dollars. If the string
contains no ammount return "Unavailable"
"""
def getMaxInUSD(fees:str, country:str):
    money = parseFees(str(fees), country)                                                             # parse the fees text 
    if len(money) == 0:return "Unavailable"                                                           # if no tuples where returned return "Unavailable"
    maxInUsd = max([(amnt/usdRates.get(code)) if code in usdRates else 0 for code, amnt in money])    # convert each ammount to USD and return max value
    return f'${maxInUsd:.2f}' if maxInUsd else "Unavailable"                                          # format the ammount to 2 digits precision + $ symbol


# Create a new filed called fees_USD where the max fees ammount in USD is stored
# to get the right value both fees and country name are used to resolve
# ambiguities inside fees string
pdf.loc[:, 'fees_USD'] = df.apply(lambda row: getMaxInUSD(row['fees'], row['country']), axis=1)

pdf


,courseName,universityName,description,city,country,fees,tokens,id,fees_USD
0,3D Design for Virtual Environments - MSc,Glasgow Caledonian University,3D visualisation and animation play a role i...,Glasgow,United Kingdom,Please see the university website for further ...,"[visualis, anim, play, role, mani, area, popul...",0,Unavailable
1,Accounting and Finance - MSc,University of Leeds,Businesses and governments rely on sound fin...,Leeds,United Kingdom,"UK: £18,000 (Total) International: £34,750 (To...","[busi, govern, reli, sound, financi, knowledg,...",1,$42542.88
2,Accounting and Finance (MSc),University of Bath,Develop in-depth knowledge of accounting and...,Bath,United Kingdom,Please see the university website for further ...,"[develop, knowledg, account, financ, theori, l...",2,Unavailable
3,"Accounting, Accountability & Financial Manag...",King’s College London,"Our Accounting, Accountability & Financial M...",London,United Kingdom,Please see the university website for further ...,"[account, account, financi, manag, msc, cours,...",3,Unavailable
4,"Accounting, Financial Management and Digital...",University of Reading,Embark on a professional accounting career w...,Reading,United Kingdom,Please see the university website for further ...,"[embark, profession, account, career, academ, ...",4,Unavailable
...,...,...,...,...,...,...,...,...,...
5995,Masters Of Finance (International Finance),Zhejiang Gongshan University,Master’s in Finance (International Finance) ...,Hangzhou,China,US$3500 per year,"[master, financ, intern, financ, zhejiang, gon...",5995,$3500.00
5996,Master's of Financial Technology (Fintech),Harbour.Space University,Harbour.Space's FinTech Master programme is ...,Barcelona,Spain,"€29,900/year","[fintech, master, programm, design, prepar, gr...",5996,$31903.30
5997,Master's of Front-end Development,Harbour.Space University,Front-end Development at Harbour.Space Unive...,Barcelona,Spain,"€29,900/year","[develop, univers, provid, uniqu, environ, stu...",5997,$31903.30
5998,Masters of Science in Business,Oregon State University,Our Master of Science in Business (MSB) will...,Corvallis,USA,You can find more information here: Please see...,"[master, scienc, busi, msb, give, focus, explo...",5998,Unavailable


<h5 style="color:green"> 2.2 Conjuctive query </h5>

<b> 2.2.1 Create your index ! </b>

Before building the index, 
* Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

Then, the first brick of your homework is to create the Inverted Index. It will be a dictionary in this format:

```
{
term_id_1:[document_1, document_2, document_4],
term_id_2:[document_1, document_3, document_5, document_6],
...}
```
where _document\_i_ is the *id* of a document that contains that specific word.

__Hint:__ Since you do not want to compute the inverted index every time you use the Search Engine, it is worth thinking about storing it in a separate file and loading it in memory when needed.

In [256]:
"""
Map function

    Create a series that for each master contains a dictionary
that has description tokens for keys and masterID for values
Ex. 
id   courseName   universityName                  ....   tokens 
0   3D desigh     Glasgow Caledonian University   ....   ['a', 'b', 'c'] 
1   Architecture  University of Bath              ....   ['e', 'a', 'd'] 

{'a': 0, 'b':0, 'c':0} 
{'e': 1, 'a':1, 'd':1} 
"""

def generateTokenDocs(row):
    docId = row['id']                               # get the document id
    uniqueTokens = set(row['tokens'])               # get the unique tokens
    return {token:docId for token in uniqueTokens}  # create a dictionary {token1: docId, token2: docId...} with same docId as value




"""
Reduce function

    Reduces two dictionaries in one single dictionary
by aggregating the values in lists


Ex.
{'a': [0,3], 'b':[0,6,7], 'c':[0,1]} 
{'e': 1, 'a':1, 'd':1} 

{'a':[0,3,1], b:[0,6,7], c:[0,1], d:[1], e:[1]}
"""
def red(prev, next):
    for k in next.keys():                         # for token in next dictionary
        if k in prev: prev[k].append(next[k])     # if token in prev append to prev[token] the docId from next[token]
        else: prev[k] = [next[k]]                 # if the token is not yet in prev create a new entry with value [docId] 
    return prev                                   # return prev dictionary




"""
    Map Reduce
    The reduce() function applies red to a list of elements in a recursive manner
    if we have reduce(red, [dict1, dict2, dict3, dict4], {})
    the result will be the same as 
    red(red(red(red({}, dict1), dict2), dict3), dict4)

    The red() function must take 2 values
    - accumulator (contains the accumulated result up to step i)
    - next        (contains the next value to apply to the result)
    The way next is applied to accumulator is defined in red() function
"""
tokenScores = functools.reduce(red, pdf.apply(generateTokenDocs, axis=1), {})


with open("index.json", "w") as fd:             # create a new file
    json.dump(tokenScores, fd)                  # save the reversed index to a file




<b> 2.2.2 Execute the query </b>

Given a query input by the user, for example:

```
advanced knowledge
```

The Search Engine is supposed to return a list of documents.

##### What documents do we want?
Since we are dealing with conjunctive queries (AND), each returned document should contain all the words in the query.
The final output of the query must return, if present, the following information for each of the selected documents:

* `courseName`
* `universityName`
* `description`
* `URL`

__Example Output__ for ```advanced knowledge```: (please note that our examples are made on a small batch of the full dataset)

<p align="center">
<img src="img/output1.png" width = 1000>
</p>

If everything works well in this step, you can go to the next point and make your Search Engine more complex and better at answering queries.


In [259]:

"""
Get the masters whose description contain
all the words in query.
"""
def queryMasters(query:[str]):
    with open('index.json') as fd:                                                              # open the index.json file
        indexDoc = json.load(fd)                                                                # load the index 
    stQuery = map(stemmer.stem, query)                                                          # stem the query tokens
    indexes = set.intersection(*[set(indexDoc[token]) for token in stQuery])                    # for each token in query get the docs that contains them and then itersect 
    return pdf.loc[pdf.id.isin(indexes)]                                                        # return the masters documents that contain all the tokens in query

queryMasters(['allergy', 'health'])

# pdf

,courseName,universityName,description,city,country,fees,tokens,id,fees_USD
1064,Allergy - MSc/PGDip/PGCert,Imperial College London,Allergy is an increasing global health probl...,London,United Kingdom,Please see the university website for further ...,"[allergi, increas, global, health, problem, ur...",1064,Unavailable
1065,Allergy & Clinical Immunology (Online) PGCert,University College Cork,This new Postgraduate Certificate in Allergy...,Cork,Ireland,"The EU fee for this course is €3,130. The Non...","[new, postgradu, certif, allergi, clinic, immu...",1065,$7415.65
2017,Clinical and Health Sciences - MSc/PGDip/PGCert,Newcastle University,Our Clinical and Health Sciences CPD module ...,Newcastle,United Kingdom,Please see the university website for further ...,"[clinic, health, scienc, cpd, modul, programm,...",2017,Unavailable


<h5 style="color:green"> 2.3 Conjunctive query & Ranking score </h5>
<br>

For the second search engine, given a query, we want to get the *top-k* (the choice of *k* it's up to you!) documents related to the query. In particular:

* Find all the documents that contain all the words in the query.
* Sort them by their similarity with the query.
* Return in output *k* documents, or all the documents with non-zero similarity with the query when the results are less than _k_. You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

To solve this task, you must use the *tfIdf* score and the _Cosine similarity_. The field to consider is still the `description`. Let's see how.


<b> 2.3.1 Inverted index </b>

Your second Inverted Index must be of this format:

```
{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}
```

Practically, for each word, you want the list of documents in which it is contained and the relative *tfIdf* score.

__Tip__: *TfIdf* values are invariant for the query. Due to this reason, you can precalculate and store them accordingly.

In [260]:
"""
    Generate tfIdf reverse index file
"""




"""
Map function

Takes in input a document and for
each token in document calculates
tfIdf(token) score
returns a dictionary of type
{
token1:[docId1, tfIdf(token1)]
token2:[docId1, tfIdf(token2)]
token3:[docId1, tfIdf(token3)]
}
"""

with open('index.json') as fd:                                                      # Open the index file
    indexDoc = json.load(fd)                                                        # loat the index


def idf(token:string):                                                              
    return math.log(len(pdf)/(len(indexDoc[token])))                                # compute log(docCount/docWithTokenCount)

def tf(text:[str], token:str):
    return text.count(token)/len(text)                                              # compute tokenCount/totTokenCount

def generateTokenDocScores(doc):
    docId = doc['id']                                                               # get the docId
    tokens = doc['tokens']                                                          # get the doc tokens
    return {token:[docId, tf(tokens, token)*idf(token)] for token in set(tokens)}   # compute tfIdf score for each token



"""
Reduce function

Reduces two dictionaries of type

{
token1:[[docId1, tfIdf(token1)] ... ]
token2:[[docId1, tfIdf(token2)] ... ]
token3:[[docId1, tfIdf(token3)] ... ]
}

{
token1:[docId2, tfIdf(token1)]
token2:[docId2, tfIdf(token2)]
token3:[docId2, tfIdf(token3)]
}

In a single dictionary of type

{
token1: [[docId1, tfIdf(token1)],..., [docId2, tfIdf(token1)]]
token2: [[docId1, tfIdf(token2)],..., [docId2, tfIdf(token2)]]
token3: [[docId1, tfIdf(token3)],..., [docId2, tfIdf(token3)]]
}

If same token is found in more then 1 dict
then the values are accumulated in a list.

....
"""

def red(prev, next):
    for k in next.keys():
        if k in prev: prev[k].append(next[k])
        else: prev[k] = [next[k]]
    return prev



"""
Map Reduce

The reduce() function applies red() to a list of elements in a recursive manner
if we have reduce(red, [dict1, dict2, dict3, dict4], {})
the result will be the same as 
red(red(red(red({}, dict1), dict2), dict3), dict4)

The red() function must take 2 values
- accumulator (contains the accumulated result up to step i)
- next        (contains the next value to apply to the result)
The way next is applied to accumulator is defined in red() function
"""
tokenScores = functools.reduce(red, pdf.apply(generateTokenDocScores, axis=1), {})


# Save the index
with open("tfIdfIndex.json", "w") as fd:
    json.dump(tokenScores, fd)


<b>2.3.2 Execute the query </b>

In this new setting, given a query, you get the proper documents (i.e., those containing all the query's words) and sort them according to their similarity to the query. For this purpose, as the scoring function, we will use the Cosine Similarity concerning the *tfIdf* representations of the documents.

Given a query input by the user, for example:
```
advanced knowledge
```
The search engine is supposed to return a list of documents, __ranked__ by their Cosine Similarity to the query entered in the input.

More precisely, the output must contain:
* `courseName`
* `universityName`
* `description`
* `URL`
* The similarity score of the documents with respect to the query (float value between 0 and 1)
  
__Example Output__ for ```advanced knowledge```:

<p align="center">
<img src="img/output2.png" width = 1000>
</p>

In [263]:

"""
    Given a query [token1, token2, ... tokenn]
The function returns the first k documents
with highest score similarity with the query
using [scoreFunction] to compute the scores.

    The similarity is calculated considering the
tfIdf score of single tokens in document and in query
"""

def idf(token:string):
    return math.log(len(pdf)/(len(indexDoc[token])))                    # compute log(docCount/docWithTokenCount)

def tf(text:[str], token:str):
    return text.count(token)/len(text)                                  # compute tokenCount/totTokenCount


def queryMasters2(query:[str], k:int, scoreFunction):

    with open('tfIdfIndex.json') as fd:                                 # get the tfIdf index 
        indexDoc = json.load(fd)                                        # parse tfIdf index to dictionary of type {token: [[docId, tfIdf(toke)], ...}


    stQuery = [token for token in list(map(stemmer.stem, query)) if token in indexDoc]  # stem the query tokens and get only the ones in indexDoc
    dimensions = len(stQuery)                                                           # get the number of dimensions of the query
    queryVector = [tf(stQuery, token)*idf(token) for token in stQuery]                  # compute the query vector (map each token to tfIdf(token) value)



    # get the ids of the documents that contain all the tokens from the query
    # other documents are not considered because their score will be 0  (because tf(token) = 0  if token not in doc)
    docIdsSet = set.intersection(*[{docId for docId, tfIdf in indexDoc[token]} for token in stQuery])
    docIds = sorted(docIdsSet)


    # Compute the documents vectors
    docVD = {docId:[0]*dimensions for docId in docIds}                   # Start with a 0-ed vector for each document (with same number of dimensions as query)
    for dimension,token in enumerate(stQuery):                           # For each (dimension, token) in query 
        for docId, docTfIfd in indexDoc[token]:                          # For each document that contains that token
            if docId in docIdsSet:                                       # If the document is among the documents that contain all the tokens in the query
                docVD[docId][dimension] = docTfIfd                       # Set the document vector dimension value to tfIdf(token)
    

    heap = []
    for docId, docVector in docVD.items():                               # Iterate on documents ids and vectors
        heappush(heap, (-scoreFunction(docVector, queryVector), docId))  # compute similarity score and push it in a min heap 
    
    

    topKIds = [] 
    scores = []
    for i in range(min(k, len(docIds))):                                 # pop k values from the heap and store the ids and the scores
        scoreDocId = heappop(heap)
        topKIds.append(scoreDocId[1])
        scores.append(-scoreDocId[0])


    newDf:pd.DataFrame = pdf.iloc[topKIds].copy()                        # create new dataframe with only the top k
    newDf.loc[:, 'Similarity'] = scores                                  # add scores to the new dataframe
    return newDf                                                         # return the k docs with highest score

   

In [267]:
 
# Returns the module of a vector
def vectorMod(v):
    return math.sqrt(sum(map(lambda x: x*x, v)))

# Given 2 vectors returns the cosine similarity 
def cosSimilarity(v1, v2):
    return sum(map(lambda x: x[0]*x[1], zip(v1,v2))) / (vectorMod(v1)*vectorMod(v2))

# Get the top k documents with highest cosineSimilarity to query 
query = ['allergi', 'health']
k = 5
queryMasters2(query, k, cosSimilarity)

,courseName,universityName,description,city,country,fees,tokens,id,fees_USD,Similarity
1065,Allergy & Clinical Immunology (Online) PGCert,University College Cork,This new Postgraduate Certificate in Allergy...,Cork,Ireland,"The EU fee for this course is €3,130. The Non...","[new, postgradu, certif, allergi, clinic, immu...",1065,$7415.65,1.000000
1064,Allergy - MSc/PGDip/PGCert,Imperial College London,Allergy is an increasing global health probl...,London,United Kingdom,Please see the university website for further ...,"[allergi, increas, global, health, problem, ur...",1064,Unavailable,0.975020
2017,Clinical and Health Sciences - MSc/PGDip/PGCert,Newcastle University,Our Clinical and Health Sciences CPD module ...,Newcastle,United Kingdom,Please see the university website for further ...,"[clinic, health, scienc, cpd, modul, programm,...",2017,Unavailable,0.967671


<hr>

<h1> 3. Define a new score </h1>
Now it's your turn: build a new metric to rank MSc degrees.

Practically:

1. The user will enter a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 2.1.
2. Once you have the documents, you need to sort them according to your new score. In this step, you won't have any more to take into account just the ```description``` field of the documents; you __can__ use also the remaining variables in your dataset (or new possible variables that you can create from the existing ones or scrape again from the original web-pages). You __must__ use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

__N.B.:__ You have to define a __scoring function__, not a filter! 

The output, must contain:

* `courseName`
* `universityName`
* `description`
* `URL`
* The  __new__ similarity score of the documents with respect to the query

Are the results you obtain better than with the previous scoring function? **Explain and compare results**.

In [268]:

# The new score calculates the distance between v1 and v2
# that is  |v1-v2|
def distanceSimilarity(v1, v2):
    return vectorMod(map(lambda x: x[0]-x[1], zip(v1,v2))) 

# Get the top k documents with highest distanceSimilarity to query 
query = ['health', 'air']
k = 5
queryMasters2(query, k, distanceSimilarity)


,courseName,universityName,description,city,country,fees,tokens,id,fees_USD,Similarity
3417,Environment and Human Health MSc/PgDip/PgCert,University of Exeter,Overview You will investigate the intricate ...,Exeter,United Kingdom,For current programme fees please see our webs...,"[overview, investig, intric, relationship, con...",3417,Unavailable,2.686758
138,Environmental Engineering and Project Manage...,University of Leeds,There are many challenges we face globally. ...,Leeds,United Kingdom,"UK: £13,750 (Total) International: £31,000 (To...","[mani, challeng, face, global, one, pertin, pr...",138,$37951.92,2.682534
3454,Environmental Engineering - MSc,Imperial College London,This course provides training in all aspects...,London,United Kingdom,Please see the university website for further ...,"[cours, provid, train, aspect, suppli, clean, ...",3454,Unavailable,2.669385
5692,Master of Science (MS) in Geography and Envi...,Johns Hopkins University,MSGEE students take classes and conduct rese...,Baltimore,USA,Please see the university website for further ...,"[msgee, student, take, class, conduct, researc...",5692,Unavailable,2.660112
4298,Health Technology - MSc,"University of the West of England, Bristol",The use of digital technology in healthcare ...,Bristol,United Kingdom,Full time Home Award Fee £9000 Home Module Fee...,"[use, digit, technolog, healthcar, grow, rapid...",4298,$11018.30,2.658761


In [269]:
# Lets compare to the cosine similarity scoring function
query = ['health', 'air']
k = 5
queryMasters2(query, k, cosSimilarity)


,courseName,universityName,description,city,country,fees,tokens,id,fees_USD,Similarity
138,Environmental Engineering and Project Manage...,University of Leeds,There are many challenges we face globally. ...,Leeds,United Kingdom,"UK: £13,750 (Total) International: £31,000 (To...","[mani, challeng, face, global, one, pertin, pr...",138,$37951.92,1.000000
3454,Environmental Engineering - MSc,Imperial College London,This course provides training in all aspects...,London,United Kingdom,Please see the university website for further ...,"[cours, provid, train, aspect, suppli, clean, ...",3454,Unavailable,1.000000
5692,Master of Science (MS) in Geography and Envi...,Johns Hopkins University,MSGEE students take classes and conduct rese...,Baltimore,USA,Please see the university website for further ...,"[msgee, student, take, class, conduct, researc...",5692,Unavailable,1.000000
4298,Health Technology - MSc,"University of the West of England, Bristol",The use of digital technology in healthcare ...,Bristol,United Kingdom,Full time Home Award Fee £9000 Home Module Fee...,"[use, digit, technolog, healthcar, grow, rapid...",4298,$11018.30,0.956165
1051,Air Pollution Management and Control - MSc/P...,University of Birmingham,Our Air Pollution Management and Control MSc...,Birmingham,United Kingdom,Please see the university website for further ...,"[air, pollut, manag, control, msc, programm, k...",1051,Unavailable,0.949304


In [241]:

"""
Conclusion:  

    The results are somewhat similar but come in slightly different
order. While cosine similarity consider the angle between vectors v1 and v2 
distance similarity considers the distance between v1 and v2 therefore in some cases
we can have 2 documents that have high similarity using cosSimilarity function
but low similarity when using distance similarity (consider 2 vectors with same direction and sense but
very different modules).

The results are definitively better with cosSimilarity
"""

'\nConclusion:  \n\n    The results are somewhat similar but come in slightly different\norder. While cosine similarity consider the angle between vectors v1 and v2 \ndistance similarity considers the distance between v1 and v2 therefore in some cases\nwe can have 2 documents that have high similarity using cosSimilarity function\nbut low similarity when using distance similarity (consider 2 vectors with same direction and sense but\nvery different modules).\n\nThe results are definitively better with cosSimilarity\n'

<hr>

<h1> 4. Visualizing the most relevant MSc degrees </h1>

<p>Using maps can help people understand how far one university is from another so they can plan their academic careers more adequately. Here, we challenge you to show a map of the courses found with the score defined in point 3. You should be able to identify at least the *city* and *country* for each MSc degree. You can find some ideas on how to create maps in Python [Here](https://plotly.com/python/maps/) and [Here](https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621) but you will maybe need further information for a proper visualization, like coordinates (latitude and longitude). You can retrieve this data using various tools:

1. [Here](https://medium.com/@manilwagle/geocoding-the-world-using-google-api-and-python-1f6b6fb6ca48) you can find a helpful tutorial on how to encode geo-informations using Google API in Python (this tool can also be used in [Google Sheets](https://handsondataviz.org/geocode.html))
2. You can collect a list of unique places in the format (City, Country) and ask chatGPT (or, as usual, any other LLM chatbot) to provide you with a list of corresponding representative coordinates
3. Explore and find the best solution for your case!
   
Once you defined your visualization strategy, include a way to encode fees in your charts. The map should show (with a proper legend) different courses and associated taxation: the user wants a glimpse not only of how far he will need to move but also of how much it will cost him! </p>

In [270]:

"""
The function returns the coordinates of a city
Given that there are cities with same names in different
countries a country input is used to select the
right one
"""

# Get coordinates of city by using geopy library
def getCoordinates(country:string, city:string):
    loc = geolocator.geocode(city+','+country) 
    return [loc.longitude, loc.latitude]

# Display a map with masters that match the query and 
# and other usefull information
def displayMasters(query, k):
    result = queryMasters2(query, k, distanceSimilarity)                                                    # get the top k masters with highest similarity score

    addresses = result.loc[:, ['courseName','universityName', 'description', 'fees_USD']].copy()            # create a new dataframe to show info in map 
    coordinates = result.apply(lambda row: getCoordinates(row['country'], row['city']), axis=1)             # find coordinates of each master
    if (coordinates.size):
        coordinates = coordinates.to_list()                                                                 
    else:
        return "NO MASTERS FOUND" 

    worldMap = KeplerGl(height=600, width=800)                                                              # create a new map
    gdf = gpd.GeoDataFrame(addresses, geometry=gpd.points_from_xy(*zip(*coordinates)))                      # use the coordinates to display masters locations
    worldMap.add_data(data=gdf, name='kepler map')                                                          

    display(worldMap)                                                                                       # display map



In [273]:
# displayMasters(['health', 'air'], 5)
queryMasters2(['health', 'air'], 5, distanceSimilarity)

,courseName,universityName,description,city,country,fees,tokens,id,fees_USD,Similarity
3417,Environment and Human Health MSc/PgDip/PgCert,University of Exeter,Overview You will investigate the intricate ...,Exeter,United Kingdom,For current programme fees please see our webs...,"[overview, investig, intric, relationship, con...",3417,Unavailable,2.686758
138,Environmental Engineering and Project Manage...,University of Leeds,There are many challenges we face globally. ...,Leeds,United Kingdom,"UK: £13,750 (Total) International: £31,000 (To...","[mani, challeng, face, global, one, pertin, pr...",138,$37951.92,2.682534
3454,Environmental Engineering - MSc,Imperial College London,This course provides training in all aspects...,London,United Kingdom,Please see the university website for further ...,"[cours, provid, train, aspect, suppli, clean, ...",3454,Unavailable,2.669385
5692,Master of Science (MS) in Geography and Envi...,Johns Hopkins University,MSGEE students take classes and conduct rese...,Baltimore,USA,Please see the university website for further ...,"[msgee, student, take, class, conduct, researc...",5692,Unavailable,2.660112
4298,Health Technology - MSc,"University of the West of England, Bristol",The use of digital technology in healthcare ...,Bristol,United Kingdom,Full time Home Award Fee £9000 Home Module Fee...,"[use, digit, technolog, healthcar, grow, rapid...",4298,$11018.30,2.658761


<img src="img/map.png">

<hr>

<h1> 6. Command Line Question </h1>

<p>As done in the previous assignment, we encourage using the command as a feature that Data Scientists must master.

Note: To answer the question in this section, you must strictly use command line tools. We will reject any other method of response. The final script must be placed in CommandLine.sh.

First, take the course_i.tsv files you created in point 1 and merge them using Linux commands (Hint: make sure that the first row containing the column names appears only once).

Now that you have your merged file named merged_courses.tsv, use Linux commands to answer the following questions:
- Which country offers the most Master's Degrees? Which city?
- How many colleges offer Part-Time education?
- Print the percentage of courses in Engineering (the word "Engineer" is contained in the course's name).

__Important note:__ You may work on this question in any environment (AWS, your PC command line, Jupyter notebook, etc.), but the final script must be placed in CommandLine.sh, which must be executable. Please run the script and include a __screenshot__ of the <ins>output</ins> in the notebook for evaluation.</p>


<hr>

<h1> 7. Algorithmic Question </h1>

Leonardo is an intern at a company. He is paid based on the total number of hours he has worked. They agreed __d__ days ago that Leonardo could not work less than $minTime_i$ or more than $maxTime_i$ hours per <ins>i-th</ins> day. Furthermore, he was warned by HR that on his last day at the company, he should provide a detailed report on how many hours he worked <ins>each day</ins> for the previous d days.

Today is the day Leonardo should report to HR, but the problem is that he <ins>didn't</ins> account for how many hours he put in for each day, so he only has the __total sum of the hours__ ($sumHours$) he put in total in these d days. He believes that if he creates a report in which each number $dayHours_i$ corresponds to the __total hours he worked on the i-th day__ while satisfying the HR limitations and the total sum of all $dayHours_i$ equals $sumHours$, he would be fine.

He cannot create such a report independently and requests your assistance. He will give you the number of days $d$, total hours spent $sumHours$, and the HR limitations for each day $i$, and he wants you to assist him in determining whether it is possible to create such a fake report. If that is possible, make such a report. 

**Input**

The first line of input contains two integers __d__, $sumHours$ - the number of days Leonardo worked there and the total number of hours he worked for the company. Each of the following __d__ lines contains two integer numbers $minTime_i$ and $maxTime_i$ - the minimum and maximum hours he can work on the $i_{th}$ day. 

**Output**

If such a report cannot be generated, print 'NO' in one output line. If such a report is possible, print 'YES' in the output and d numbers - the number of hours Leonardo spent each day - in the second line. If more than one solution exists, print any of them. 

__Input 1__
```
2 5
0 1
3 5
```
__Output 1__
```
YES
1 4 
```
__Input 2__
```
1 1
5 6
```
__Output 2__
```
NO
```

1. Implement a code to solve the above mentioned problem. 

2. What is the __time complexity__ (the Big O notation) of your solution? Please provide a <ins>detailed explanation</ins> of how you calculated the time complexity.

3. Ask ChatGPT or any other LLM chatbot tool to check your code's time complexity (the Big O notation). Compare your answer to theirs. Do you believe this is correct? If the <ins>two differ</ins>, which one is right? (why?)
   
4. What do you think of the __optimality__ of your code? Do you believe it is optimal? Can you improve? Please <ins>elaborate</ins> on your response. 



In [244]:

"""
Let's say the function takes input the following input
d, sumHours
minTime1 maxTime1
minTime2 maxTime2
minTime3 maxTime3
minTime4 maxTime4
....

The hours in the report will then
be

minTime1 < workTime1 < maxTime1
minTime2 < workTime2 < maxTime2
minTime3 < workTime3 < maxTime3
minTime4 < workTime4 < maxTime4
....

so for the report we need that
sum(minTime1 ...) < sumHours < sum(maxTime1 ...)

otherwise we cannot create the report 
either because we don't have enought hours to match the min values
either because we have too many hours to stay within the max hours limits
In both cases we return NO

If instead sum(minTime1 ...) < sumHours < sum(maxTime1 ...) then
We use the following routine to distribute hours
    1. dayHours = [minTime1, minTime2, minTime3, ....]  set the initial day hours to minimum
    2. excessHours = sumHours - sum(minTime1 ...) to get the excess hours (the hours that go over the minimum threshold)
    3. Keep an index i to access the hours of the i-th day (that for each loop iterates inside [0, d-1] module)
    4. Distribute uniformly the excessHours over the d days in an iterative way (considering the max tresholds)
    5. While there are any excessHours  (excessHours != 0) 
        - get the i-th day hours
        - add 1 to minTimeX and subtract 1 to excessHours if minTimeX < maxTime (do not exceed the max treshold)
    5. return dayHours 
"""

def getReport(d, sumHours, times):
    totMin, totMax = map(sum, zip(*times))              # Get the totMin = sum([minTime1 ... minTimeN]) totMax = sum([maxTime1 ... maxTimeN])
    if totMin <= sumHours and sumHours <= totMax:       # Check if the sumHours can be distributed in the report
        print("YES")                                    # If true then print "YES"
        dayHours = [minTime for minTime, _ in times]    # Set the initial report to all minimal hours
        excessHours = sumHours - totMin                 # Get the excessHours
        i = 0                                           # Set an index to access day i hours
        while(excessHours):                             # While there are excessHours to distribute
            maxDayHours = times[i][1]                   # Get the maxHours for day i
            if dayHours[i] < maxDayHours:               # See if we can add an extra hour to day i
                dayHours[i] += 1                        # If yes add an extra hour to the day 1 
                excessHours -= 1                        # Subtract an extra hour to excessHours
            i = (i+1)%d                                 # go to next day (if last day go to first)
        print(" ".join(map(str, dayHours)))             # print report 
    else:
        print('NO')                                     # if sumHours no in range of totMin totMax print NO

# d, sumHours = map(int, input().split())
# times = [list(map(int, input().split())) for _ in range(d)]
# getReport(d, sumHours, times)

# getReport(2, 5, [[0,1], [3,5]])
# getReport(1, 1, [[5,6]])
getReport(5, 24, [[0,1], [2,3], [5,8], [4,7], [2,5]])


YES
1 3 8 7 5


In [245]:
"""
Complexity

    Worst case
Let's consider the worst case scenario 

sum(minTime1 ...) < sumHours < sum(maxTime1 ...)

1. [0, m] 
2. [0, 0]
3. [0, 0]
4. [0, 0]
i. .... 
d. [0, 0]

excessHours = sumHours


In this case we decrement excessHours by 1 for one day and do nothing for the next d-1 days with cost O(d)
and we must do this sumHours times (until sumHours == 0) with cost O(sumHours*d)

In worst case scenario the complexity is O(sumHours*d)

    

    Best case
The best case scenario is when the report can't be created 
sum(minTime1 ...) < sumHours < sum(maxTime1 ...)  == False
We still must spend O(d) to calculate totMin a totMax
but the if check is constant

In best case scenario the complexity is O(d) 

"""

def getReport(d, sumHours, times):
    totMin, totMax = map(sum, zip(*times))               # O(d) iterates constant times on array with len = d 
    if totMin < sumHours and sumHours < totMax:          # O(1)  constant
        print("YES")                                     # O(1) 
        dayHours = [minTime for minTime, _ in times]     # O(d)
        excessHours = sumHours - totMin                  # O(1)
        i = 0                                            # O(1)
        while(excessHours):                              # O(sumHours*d)  worst case scenario
            maxDayHours = times[i][1]                    # O(sumHours*d)
            if dayHours[i] < maxDayHours:                # O(sumHours*d) 
                dayHours[i] += 1                         # O(sumHours) 
                excessHours -= 1                         # O(sumHours)
            i = (i+1)%d                                  # O(sumHours*d)
        print(" ".join(map(str, dayHours)))              # O(d) 
    else:                                                # O(1)
        print('NO')                                      # O(1) 



In [246]:
"""
The above code is not optimal, we can improve it
by distributing the excessHours in a non uniform way
the first days get the maximum hours while the last
days get the minimum values


"""
def getReport(d, sumHours, times):
    totMin, totMax = map(sum, zip(*times))              # Get the totMin = sum([minTime1 ... minTimeN]) totMax = sum([maxTime1 ... maxTimeN])
    if totMin <= sumHours and sumHours <= totMax:       # Check if the sumHours can be distributed in the report
        print("YES")                                    # If true then print "YES"
        dayHours = [minTime for minTime, _ in times]    # Set the initial report to all minimal hours
        excessHours = sumHours - totMin                 # Get the excessHours
        for i in range(d):                              # Iterate on days
            minDayHours = times[i][0]                   # get the day i min hours
            maxDayHours = times[i][1]                   # get the day i max hours
            delta = maxDayHours - minDayHours           # get the hours we can add to the day i to reach max hours
            if delta <= excessHours:                    # if we can get all the hours from excessHours
                excessHours -= delta                    # take the hours from excessHours
                dayHours[i] += delta                    # add the hours the the day i
            else:                                       # if we cannot get all the hours from excessHours 
                dayHours[i] += excessHours              # get remaining hours in excessHours and add them to day i 
                break                                   # break from the loop because excessHours are 0
        print(" ".join(map(str, dayHours)))             # print report 
    else:
        print('NO')                                     # if sumHours no in range of totMin totMax print NO


getReport(5, 19, [[0,1], [2,3], [5,8], [4,7], [2,5]])

YES
1 3 8 5 2


In [247]:
"""
Complexity
In this solution we iterate on days only once
all the costs inside the for loop are constant so
the complexity is O(d) in worst case scenario.

Worst case scenario is when we need to add extra hours to every
day not just the first ones .

The cost in worst case scenario is O(d)

Best case scenario is when the report cannot be created
just like before the cost is O(d) to calculate totMin and totMax

The cost in best case scenario is O(d)


This is the optimal solution because in worst case scenarios
we must divide the excessHours amongst all days
so we must consider all the days in the report that
has length d. So the problem itself has complexity Omega(d)

"""

def getReport(d, sumHours, times):
    totMin, totMax = map(sum, zip(*times))               # O(d)
    if totMin <= sumHours and sumHours <= totMax:        # O(1) 
        print("YES")                                     # O(1)
        dayHours = [minTime for minTime, _ in times]     # O(d)
        excessHours = sumHours - totMin                  # O(1)
        for i in range(d):                               # O(d)
            minDayHours = times[i][0]                    # O(d) 
            maxDayHours = times[i][1]                    # O(d) 
            delta = maxDayHours - minDayHours            # O(d) 
            if delta <= excessHours:                     # O(d) 
                excessHours -= delta                     # O(d) 
                dayHours[i] += delta                     # O(d) 
            else:                                        # O(d) 
                dayHours[i] += excessHours               # O(d) 
                break                                    # O(d) 
        print(" ".join(map(str, dayHours)))              # O(d) 
    else:
        print('NO')                                     


<hr>